Моё решение - это немгого улучшенный бейзлайн.
Если почитать статью https://habr.com/ru/company/vk/blog/552162/, то можно подчерпнуть следующие идеи:
- Для каждой пары (`v1`, `v2`) посчитаем `score(v1, v2)` - чем больше это число, тем вероятнее, что `v1` и `v2` подружатся. Для начала - это количество общих друзей.
- Чем больше друзей у `v`, тем менее вероятно, что 2 случайных его друга подружатся. Поэтому теперь вес у общего друга `v1` и `v2` не `1`, а `1 / log(N(v))`, где `N(v)` - количество друзей `v`. 
- Нужно как-то использовать величину `h` — активность взаимодействия между пользователями. Чем больше активность - тем больше шансов этого друга с кем-то сдружить. Теперь скор будет `log((h1+1)*(h2+1)) / log(N(v))`
- Нужно как-то использовать величину `t` - время, прошедшее с момента дружбы. Если 2 друга подружились давно, то, скорее всего, общие друзья у них уже устоялись. Для простоты поштрафуем `h` на величину `log(t + 1) - 2`. Если дружат давно с высоким `h`, то этот `h` будет чуть меньше влиять.
Итоговая формула выйдет: `log((H1+1)*(H2+1)) / log(N(v))`,
где `H1 = max(0, h1 - max(0, log(t1 + 1) - 2))` и `H2 = max(0, h2 - max(0, log(t2 + 1) - 2))` 

In [2]:
from itertools import groupby
from collections import defaultdict

import pandas as pnd
import tqdm
import numpy as np
import gc
import zipfile
import math

In [3]:
def load_graph(train_graph, train_future_friends):
    future = pnd.read_csv(train_future_friends)
    future_graph = defaultdict(list)
    for row in tqdm.tqdm(future.itertuples(), total=len(future)):
        future_graph[row.u].append(row.v)
        future_graph[row.v].append(row.u)

    for arr in future_graph.values():
        arr.sort()
    
    train = pnd.read_csv(train_graph)
    graph = defaultdict(list)
    for row in tqdm.tqdm(train.itertuples(), total=len(train)):
        graph[row.u].append((row.v, row.t, row.h))
        graph[row.v].append((row.u, row.t, row.h))
        
    for arr in graph.values():
        arr.sort()
        
    return graph, future_graph

In [4]:
graph, future_graph = load_graph('~/Downloads/2021 VK Cup ML Qual/train.csv', '~/Downloads/2021 VK Cup ML Qual/val.csv')

100%|██████████| 17414510/17414510 [00:59<00:00, 290762.93it/s]


In [5]:
graph_bool = set((id1, id2) for id1, d in graph.items() for id2, _, _ in d)
future_graph_bool = set((id1, id2) for id1, d in future_graph.items() for id2 in d)

In [6]:
def solution(sample_id, sample_size, source_id, source_size, target_id, target_size):
    score = defaultdict(lambda: defaultdict(float))
    for id, lst in tqdm.tqdm(graph.items(), total=len(graph), position=0, leave=True):
        den = math.log(len(lst))

        h_scorer = lambda h, t: max(0, h - max(0, math.log(t + 1) - 2))
        for v1, t1, h1 in lst:
            if v1 % sample_size != sample_id:
                continue
            if v1 % source_size != source_id:
                continue

            h1 = h_scorer(h1, t1)
            for v2, t2, h2 in lst:
                if v2 <= v1:
                    continue
                if v2 % target_size != target_id:
                    continue
                    
                h2 = h_scorer(h2, t2)
                    
                score[v1][v2] += ( math.log((h1+1)*(h2+1)) ) / den

    return score

In [7]:
def get_recall(res, future_graph_bool):
    cnt = 0
    for id1, lst in res.items():
        for id2 in lst:
            if (id1, id2) in future_graph_bool:
                cnt += 1
    return cnt

In [8]:
def get_result(score):
    result = {}
    for id, dct in tqdm.tqdm(score.items(), total=len(score)):
        vals = list((id2, sc) for id2, sc in dct.items() if (id, id2) not in graph_bool)
        vals.sort(key=lambda x: -x[1])
        vals = vals[0:10]
        list_str = ','.join(str(id) for id, score in vals)
        result[id] = tuple(id for id, score in vals)
    return result
    

In [9]:
# Провалидируем решение с четными ID, для которых у нас есть ответы
# Для ускорения подсчёта возьмем только 1/13 выборки
gc.collect()
score_train = solution(7, 13, 0, 2, 0, 2)
res = get_result(score_train)
get_recall(res, future_graph_bool)

100%|██████████| 113420/113420 [00:23<00:00, 4808.70it/s] 


91

In [10]:
# Получили 91 попугай, неплохо

In [11]:
# Так же, как и в бейзлайне, решаем задачу батчами, т.к. на компе мало памяти :(
result = {}
for u_part in range(5):
    score = solution(u_part, 5, 1, 8, 1, 2)
    print(f"u_part={u_part}: {len(score)}")
    res = get_result(score)
    result.update(res)

  0%|          | 19/73391 [00:00<08:33, 142.81it/s]

u_part=0: 73391


  0%|          | 14/73202 [00:00<08:49, 138.35it/s]

u_part=1: 73202


  0%|          | 0/73248 [00:00<?, ?it/s]

...

  0%|          | 0/73668 [00:00<?, ?it/s]

u_part=3: 73668


  0%|          | 12/73868 [00:00<13:46, 89.33it/s]

u_part=4: 73868


100%|██████████| 73868/73868 [00:17<00:00, 4256.46it/s] 


In [12]:
res_path = '/Users/tyamgin/Projects/mlbootcamp/vkcup2021_1/res'
with open(f"{res_path}/res.txt", 'w') as out:
    out.write("\n".join(str(id) + ": " + ",".join(map(str, lst))
                        for id, lst in result.items() 
                        if len(lst) > 0) + "\n")

with zipfile.ZipFile(f"{res_path}/res.zip", "w") as zf:
    zf.write(f"{res_path}/res.txt", "res.txt")